# Training file

This file regroups every kind of test for model training : 
- The first file & library imports are mandatory
- The last exports are mandatory
- The model training depends on the choice of algorithm

There is an option to automate the training using togglable parameters 
Make sure to have set up the correct file structure using env_setup.ipynb

## Python libraries

In [ ]:
# Data Libraries
import pandas as pd
import os

# TensorFlow Part
import tensorflow as tf
from tensorflow import keras
from keras import backend
from keras.models import Sequential
from keras.layers import Input,Dense
import keras_tuner as kt
from sklearn.model_selection import train_test_split

## Parameters

### Code parameters

In [ ]:
# Basic file constant parameters
FILENAME="model_training.ipynb"
DIRNAME=os.path.abspath(FILENAME).replace(FILENAME,'')

# JSON Automatic Handling Switch
IS_JSON_SUPPORTED = False

# Variable parameters (from JSON file)
DATA_FOLDER = "versuch_f1"
VERSION_NB = 1

DF_POINTS_RANGE = 11
DF_POINTS_LENGTH = 3000

OFFSET_XYZ=[-578.6,261.4,-375]
CARTESIAN_COLUMNS=['curCart_x','comdCart_x','curCart_y','comdCart_y','curCart_z','comdCart_z']

TEST_TRAIN_RATIO = 0.1
VALID_TRAIN_RATIO = 0.2

MAX_TRIALS = 5 
MAX_EXECUTIONS = 3
MAX_EPOCHS = 50

### JSON Support

In [ ]:
def JSON_import():

## Import training DataFrame

### Import from Feather file

In [ ]:
def data_import():
    # Load DataFrame
    full_df=pd.read_feather(DIRNAME+"data/feather/"+DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)+".feather")

    # Drop unnecessary columns
    full_df=full_df.drop(["t_Sec","t_nSec","Fx1","Fy1","Fz1","Tx1","Ty1","Tz1"],axis=1)

    # Transform position from absolute to relative
    for i in range(len(CARTESIAN_COLUMNS)):
        full_df[CARTESIAN_COLUMNS[i]] = full_df[CARTESIAN_COLUMNS[i]] + OFFSET_XYZ[i//2]

    return full_df

### Split the dataset into train/validation sets

In [ ]:
def data_split():
    # Shuffle DataFrame
    shuffled_df=full_df.sample(frac=1,random_state=1)

    # X & Y datasets
    X = shuffled_df[['exT_A1','exT_A2','exT_A3','exT_A4','exT_A5','exT_A6','exT_A7',
                    'msT_A1','msT_A2','msT_A3','msT_A4','msT_A5','msT_A6','msT_A7',
                    'Fx','Fy','Fz','Tx','Ty','Tz']].to_numpy()

    # Regression Y
    Y = shuffled_df[['curCart_x','curCart_y','curCart_z']].to_numpy()
    # Classification Y


    # Train / Test / Validation dataset
    X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size=TEST_TRAIN_RATIO, random_state=2)
    X_Train, X_Valid, Y_Train, Y_Valid = train_test_split(X_Train, Y_Train, test_size=VALID_TRAIN_RATIO, random_state=3)

    return X_Train,X_Test,X_Valid,Y_Train,Y_Test,Y_Valid

## Model building

### Model 1 - FFNN Regression

In [ ]:
class FFNN_Regression_Model(kt.HyperModel):

    # Search metrics
    objective="mae"

    def rmse(y_true,y_pred):    # Private
        return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))
    
    def rsquared(y_true,y_pred):    # Private
        RSS =  backend.sum(backend.square( y_true- y_pred ))
        TSS = backend.sum(backend.square( y_true - backend.mean(y_true)))
        return 1-(RSS/TSS)

    def build_model(hp):    # Main
        # Create model
        model = Sequential()

        # Input Layer
        model.add(Dense(20, input_shape=(20,), activation='relu'))

        # Tune number of layers
        for i in range(hp.Int("num_layers", 1, 5)):
            model.add(Dense(
                    # Tune number of units
                    units = hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                    # Tune activation function
                    activation = hp.Choice("activation", ["relu","tanh"])
                            )
                    )
        # Tune whether to use dropout
        if hp.Boolean("dropout"):
            model.add(Dropout(rate=0.2))

        # Output Layer
        model.add(Dense(3, activation='linear'))

        # Define the optimizer learning rate as hyperparameter
        lr = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        # Compile model
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr), loss="mse", metrics=["mae",rmse,rsquared])
        return model

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 2 - CNN Regression

In [ ]:
class CNN_Regression_Model(kt.HyperModel):

    # Search metrics
    objective="mae"

    def build_model(hp)

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 3 - FFNN Classification

In [ ]:
class FFNN_Classification_Model(kt.HyperModel):

    # Search metrics
    objective=

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

### Model 4 - CNN Classification

In [ ]:
class CNN_Classification_Model(kt.HyperModel):

    # Search metrics
    objective=

    # Custom fit function
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(*args,batch_size=hp.Choice("batch_size", [16, 32, 64]),**kwargs,)

## Hyperparameter optimization

### Algorithm 1 - RandomSearch

In [ ]:
def KT_random_search(Selected_Model):
    tuner = kt.RandomSearch(
        Selected_Model,
        objective=Selected_Model.objective,
        max_trials=MAX_TRIALS,
        executions_per_trial=MAX_EXECUTIONS,
        overwrite=True,
        directory=DIRNAME+"/results/search",
        project_name=DATA_FOLDER+"_"+str(DF_POINTS_LENGTH)+"_"+str(VERSION_NB)
    
    best_model=tuner.get_best_models(num_models=2)[0]
    best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
    return best_model,best_hps
)

### Algorithm 2 - GridSearch

In [ ]:
def KT_GridSearch(Selected_Model):

### Algorithm 3 - Bayesian Optimizer

In [ ]:
def KT_BayesianSearch(Selected_Model):

## Model saving

In [ ]:
def Model_Save():
    # Save model to file
    # Save HPs to JSON

## Global Search

In [ ]:
def model_selector():
    if 0 :
        FFNN_Regression_Model
    elif 1 :
        CNN_Regression_Model
    elif 2 :
        FFNN_Classification_Model
    elif 3 :
        CNN_Classification_Model

    return model

In [ ]:
# Loop

# Import
# Prepare
# Split

# Model selector
# Model

# HP

# Search selector
# Search

# Save